<a href="https://colab.research.google.com/github/tmoj1428/traditional-extractive-summerization-jacard/blob/main/traditional_extractive_summerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hazm
!pip install rouge
from  __future__ import unicode_literals
from hazm import *
from xml.dom import minidom
from rouge import Rouge
from bs4 import BeautifulSoup
from google.colab import files
from pathlib import Path

     |████████████████████████████████| 317kB 6.5MB/s 
     |████████████████████████████████| 235kB 10.2MB/s 
     |████████████████████████████████| 1.4MB 11.2MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154154 sha256=2cbf0a29ffc0d8523815cbbee522da7027c2f047126956685ccb7368952f328d
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394471 sha256=69c484e55c624ded4f75ad2a3baf6a1b9f5721c4429453b452eadd75cd0cf51a
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [43]:
# initialization
class jacardSentSumm(object):
  def __init__(self, text, fileName, summerizeLen = 10):
    normalizer = Normalizer()
    normalizeText = normalizer.normalize(text)
    self.sentToken = sent_tokenize(normalizeText)
    self.wordToken = word_tokenize(normalizeText)
    self.lenPointDict = {}
    self.summerizeLen = summerizeLen
    self.fileName = fileName
    self.text = ""
    self.max_sent_length()

  def max_sent_length(self):
    for i in self.sentToken:
      self.sentWords = word_tokenize(i)
      self.lenPointDict[i] = len(self.sentWords)
    maxLen = max(self.lenPointDict.values())
    return self.index(maxLen)

  # length index and position index and jacard index
  def index(self, maxLen):
    pointDict = {}
    for i in self.sentToken:
      self.mainSent = word_tokenize(i)
      self.point = []
      for j in self.sentToken:
        if (i != j):
          self.compSent = word_tokenize(j)
          self.a = set(self.mainSent)&set(self.compSent)
          self.comSent = sorted(self.a, key = lambda k : self.mainSent.index(k))
          self.point.append(len(self.comSent) / ((len(self.mainSent) + len(self.compSent)-len(self.comSent))))
      pointDict[i] = [0.15 * (len(self.mainSent) / maxLen), 0.3 * (1/(self.sentToken.index(i)+1)), 0.55 * (sum(self.point) / len(self.point))]
    return self.calculate_final_point(pointDict)

  # calculate final point
  def calculate_final_point(self, pointDict):
    finalPoint = {}
    for i in pointDict:
      point = sum(pointDict[i])
      finalPoint[i] = point
    return self.choose_sent(finalPoint)

  # choose sentence
  def choose_sent(self, finalPoint):
    position = {}
    self.text = ""
    for i in range(self.summerizeLen):
      self.finalSent = max(finalPoint.values())
      self.keys = [k for k, v in finalPoint.items() if v == self.finalSent]
      position[self.keys[0]] = self.sentToken.index(self.keys[0])
      finalPoint.pop(self.keys[0])
    for i in range(self.summerizeLen):
      sent = min(position.values())
      key = [i for i, j in position.items() if j == sent]
      self.text += key[0]
      position.pop(key[0])
    f = open('/content/drive/My Drive/NLP/' + self.fileName[:-1] + '.txt', "a")
    f.write(self.text)
    f.close()

  def rouge_point(self, mainText, compText):
    rouge = Rouge()
    scores = rouge.get_scores(mainText, compText)
    # print(scores)
    file = open('/content/drive/My Drive/NLP/rougeResult.txt', "a")
    file.write(self.fileName[:-1] + ': ' + str(scores) + '\n')
    file.close()
    return scores

  def rouge_average(self, pointResult):
    rouge1f = rouge1p = rouge1r = rouge2f = rouge2p = rouge2r = rougelf = rougelp = rougelr = 0
    rougeAverageResult = {}
    resultLen = len(pointResult)
    for i in pointResult:
      rouge1Val = i[0].get('rouge-1')
      rouge1f += rouge1Val.get('f')
      rouge1p += rouge1Val.get('p')
      rouge1r += rouge1Val.get('r')

      rouge2Val = i[0].get('rouge-2')
      rouge2f += rouge2Val.get('f')
      rouge2p += rouge2Val.get('p')
      rouge2r += rouge2Val.get('r')

      rougelVal = i[0].get('rouge-l')
      rougelf += rougelVal.get('f')
      rougelp += rougelVal.get('p')
      rougelr += rougelVal.get('r')
    rougeAverageResult['rouge-1'] = {'f': rouge1f / resultLen, 'p': rouge1p / resultLen, 'r': rouge1r / resultLen}
    rougeAverageResult['rouge-2'] = {'f': rouge2f / resultLen, 'p': rouge2p / resultLen, 'r': rouge2r / resultLen}
    rougeAverageResult['rouge-l'] = {'f': rougelf / resultLen, 'p': rougelp / resultLen, 'r': rougelr / resultLen}

    file = open('/content/drive/My Drive/NLP/rougeAvgResult.txt', "a")
    file.write(str([rougeAverageResult]))
    file.close()
    return [rougeAverageResult]

In [4]:
def set_path(line):
  path = '/content/drive/My Drive/NLP/summDataSet/1/'+ line
  if Path(path + '.E.S.A.F.C' + '.xml').is_file():
    path1 = '/content/drive/My Drive/NLP/summDataSet/1/'+ line + '.E.S.A.F.C' + '.xml'
  elif Path(path + '.E.S.c.m.c' + '.xml').is_file():
    path1 = '/content/drive/My Drive/NLP/summDataSet/1/'+ line + '.E.S.c.m.c' + '.xml'
  else:
    path1 = '/content/drive/My Drive/NLP/summDataSet/1/'+ line + '.E.S.e.m.c' + '.xml'
  try:
    with open(path1, 'r') as f:
      data = f.read()
  except:
    print("File not accessible!")
    data = ""
  return data

In [44]:
if __name__ == '__main__':
  dataPath = '/content/drive/My Drive/NLP/Index.txt'
  summerizeLen = input("Enter summrization length:")
  rougeResult = []
  with open(dataPath, 'r') as data:
    lines = data.readlines()
    lastLine = lines[-1]
    for line in lines:
      if(line != lastLine):
        mydoc = minidom.parse('/content/drive/My Drive/NLP/dataSet/'+ line[:-1] + '.xml')
        data = set_path(line[:-1])
      else:
        mydoc = minidom.parse('/content/drive/My Drive/NLP/dataSet/' + line + '.xml')
        data = set_path(line)
      items = mydoc.getElementsByTagName('TITLE')
      items1 = mydoc.getElementsByTagName('SUMMARY')
      items2 = mydoc.getElementsByTagName('TEXT')
      if (len(items1[0].childNodes) != 0):
        sentSummerize = jacardSentSumm(items[0].childNodes[0].data + '.' + items1[0].childNodes[0].data + ' .' + items2[0].childNodes[0].data, line, int(summerizeLen))
      else:
        sentSummerize = jacardSentSumm(items[0].childNodes[0].data + '.' + items2[0].childNodes[0].data, line, int(summerizeLen))
      if (len(data) != 0):
        rougeData = sentSummerize.rouge_point(data, sentSummerize.text)
        rougeResult.append(rougeData)
    averageResult = sentSummerize.rouge_average(rougeResult)
    print(averageResult)

Enter summrization length:5
File not accessible!
[{'rouge-1': {'f': 0.4149564608117956, 'p': 0.5083667278188627, 'r': 0.3771572091588319}, 'rouge-2': {'f': 0.24824788914904122, 'p': 0.31603580595321623, 'r': 0.21686245220052566}, 'rouge-l': {'f': 0.3842722562555461, 'p': 0.4404523305555738, 'r': 0.3579917774657836}}]
